In [9]:
from dotenv import load_dotenv

load_dotenv()
import nest_asyncio
import os

nest_asyncio.apply()
import json
import base64
import moviepy
import numpy as np


from moviepy.editor import (
    VideoFileClip,
    AudioFileClip,
    ImageClip,
    CompositeVideoClip,
    CompositeAudioClip,
    ColorClip,
    concatenate_videoclips,
)

In [10]:
with open("data/data.json") as f:
    data = json.load(f)
print(data["pictures"])

[{'description': 'A crowded Sweetgreen shelf with salads organized into 26 labeled sections', 'timestamp': 0}, {'description': "Close-up of a salad being placed in the 'A' section of the shelf", 'timestamp': 8}, {'description': "Timelapse of someone's hand quickly grabbing a salad from a letter section", 'timestamp': 15}, {'description': "The 'D' section of the shelf overflowing with salads, with salads falling onto the floor", 'timestamp': 22}]


In [11]:
from moviepy.editor import VideoFileClip, vfx
import numpy as np


def create_circular_mask(clip, radius=None):
    """
    Applies a circular mask to the given clip, making the exterior of the circle transparent.
    """
    if radius is None:
        radius = min(clip.size) // 2

    def mask_frame(frame):
        h, w = frame.shape[:2]
        Y, X = np.ogrid[:h, :w]
        center = (h // 2, w // 2)
        dist_from_center = np.sqrt((X - center[1]) ** 2 + (Y - center[0]) ** 2)

        mask = dist_from_center <= radius
        new_frame = frame.copy()
        for i in range(3):  # Apply mask to each channel
            new_frame[:, :, i] = frame[:, :, i] * mask

        return new_frame

    masked_clip = clip.fl_image(mask_frame)

    # Create a mask clip
    mask_clip = clip.fl_image(lambda frame: 255 * (mask_frame(frame) > 0))
    masked_clip = masked_clip.set_mask(mask_clip.to_mask())

    return masked_clip

In [12]:
image_clips = []
pictures = data["pictures"]

# Load the headshot video to determine its duration
headshot_clip = VideoFileClip("data/headshot.mp4").resize(
    height=500
)  # Adjust resizing as needed
headshot_duration = headshot_clip.duration
headshot_audio = headshot_clip.audio

# Make audio slightly louder
headshot_audio = headshot_audio.volumex(1.5)

# Apply circular mask and position the headshot clip
headshot_clip = create_circular_mask(headshot_clip)
headshot_clip = headshot_clip.set_position(("right", "bottom")).margin(
    right=50, bottom=50, opacity=0
)

# Initialize the list for image clips
image_clips = []

for i, picture in enumerate(pictures):
    img_path = f"data/images/image_{i}.png"
    img_clip = ImageClip(img_path)

    # Resize the image to fit the width of the canvas
    img_clip = img_clip.resize(width=1080)

    # Create a black background clip with the same size as the canvas
    black_bg = ColorClip(size=(1080, 1920), color=(0, 0, 0))

    # Composite the image clip onto the black background clip
    img_clip = CompositeVideoClip(
        [black_bg, img_clip.set_position("center")], size=(1080, 1920)
    )

    # Calculate the duration each image should be displayed
    duration = picture["timestamp"] - (pictures[i - 1]["timestamp"] if i > 0 else 0)
    img_clip = img_clip.set_duration(duration)

    # Set the start time for each image based on the timestamp
    if i > 0:
        img_clip = img_clip.set_start(pictures[i - 1]["timestamp"])

    image_clips.append(img_clip)


# Concatenate image clips
video_clip = concatenate_videoclips(image_clips, method="chain")

# Adjust the final image clip to match the headshot video's duration if necessary
if video_clip.duration < headshot_duration:
    # Extend the last clip
    last_clip = image_clips[-1].set_end(headshot_duration)
    image_clips[-1] = last_clip
    video_clip = concatenate_videoclips(
        image_clips, method="chain"
    )
elif video_clip.duration > headshot_duration:
    # Truncate the video_clip to match the headshot_duration
    video_clip = video_clip.subclip(0, headshot_duration)

# Load and adjust the background music
background_music = AudioFileClip("data/music.mp3")
if background_music.duration > headshot_duration:
    background_music = background_music.subclip(0, headshot_duration)
else:
    # Loop the music if it is shorter than the headshot video
    background_music = background_music.loop(duration=headshot_duration)

final_audio = CompositeAudioClip([headshot_audio, background_music])

# Create the final composite clip
final_clip = CompositeVideoClip(
    [
        video_clip.set_duration(headshot_duration),
        headshot_clip.set_duration(headshot_duration)
    ],
    size=(1080, 1920),
).set_audio(final_audio)

# Write the final video to a file
final_clip.write_videofile("data/final_video.mp4", threads=8, fps=24)

Moviepy - Building video data/final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video data/final_video.mp4



Moviepy - Done !
Moviepy - video ready data/final_video.mp4
